In [1]:
import torch
from SimAttention.dataloader import ModelNetDataSet

root = '/home/akira/下载/Pointnet2_PyTorch-master/byol_pcl/data/modelnet40_normal_resampled'

## Dataloader test
- input: root(file path)

In [2]:
data = ModelNetDataSet(root)
DataLoader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True)
flag = 3
i = 0

for point in DataLoader:
    print(point.shape)
    i += 1
    if i == flag:
        break

9843 point clouds found.
torch.Size([4, 10000, 3])
torch.Size([4, 10000, 3])
torch.Size([4, 10000, 3])


### 可视化函数

In [3]:
import open3d as o3d

def visualize_samples(d_sample):
    print("Points in downsample set: ", len(d_sample))
    # d_sample = d_sample.numpy()
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import numpy

result = 0
for point in DataLoader:
    result = point
    break
    
print(result.shape, type(result))
demo = result[0].reshape(-1, 3)
print(demo.shape)

torch.Size([4, 10000, 3]) <class 'torch.Tensor'>
torch.Size([10000, 3])


In [5]:
demo = demo.numpy()
# visualize_samples(demo)

### 出现问题
open3d中Unable to cast Python instance to C++ type (compile in debug mode for details)；
- 网址：https://pythonmana.com/2021/09/20210921094233096q.html
- 解释：就是shape不对，需要reshape一下就好了；
- 自己：改了下可视化函数中的数据类型，使得直接输入torch类型的数据也可以输出。
- 但是：这里出来的都是torch？还是numpy会比较好？

---

查看了dataloader的源代码，发现开始是没有写torch的，也就是在包装Dataloader的时候，使用的是torch。
应该是在这里进行了转换，所以，应该需要一个numpy的dataloader吧？

测试看看。

没有numpy的，但是有一个办法就是在用的时候就是直接numpy转换，比如下面这个。
https://blog.csdn.net/guyuealian/article/details/88343924

好了，现在的逻辑就是从Dataloader读取出来的数据，首先转换成numpy 然后再使用。

读取进来之后的三个函数测试，random_point_dropout，random_scale_point_cloud，shift_point_cloud
首先还是回去看这三个函数的原函数，看看是torch的还是numpy的，看来，应该就是numpy的。
开始测试。

In [6]:
# input dim: [B,N,C]
# get the first batch point set
print(result.shape, type(result))
data_in = result.numpy()
print(data_in.shape, type(data_in))

torch.Size([4, 10000, 3]) <class 'torch.Tensor'>
(4, 10000, 3) <class 'numpy.ndarray'>


In [7]:
from SimAttention.utils.provider import random_point_dropout
points = random_point_dropout(data_in)
print(points.shape, type(points))

(4, 10000, 3) <class 'numpy.ndarray'>


In [8]:
from SimAttention.utils.provider import random_scale_point_cloud
points = random_scale_point_cloud(points)
print(points.shape, type(points))

(4, 10000, 3) <class 'numpy.ndarray'>


In [9]:
from SimAttention.utils.provider import shift_point_cloud
points = shift_point_cloud(points)
print(points.shape, type(points))

(4, 10000, 3) <class 'numpy.ndarray'>


### 测试augmentation方法的输出维度

出现问题，原始的这个是解决一个的，而不是一个batch，所以需要修改

In [10]:
from SimAttention.network.augmentation import PointWOLF
aug_method = PointWOLF()

for i in range(points.shape[0]):
    ori, aug = aug_method(points[i].reshape(-1, 3))
    # print(type(aug))
    print(aug.shape, type(aug))

(10000, 3) <class 'numpy.ndarray'>
(10000, 3) <class 'numpy.ndarray'>
(10000, 3) <class 'numpy.ndarray'>
(10000, 3) <class 'numpy.ndarray'>


In [13]:
for i in range(0, 4):
    result_original[i] = numpy.zeros((5, 3))
print(result_original.shape)

(4, 5, 3)


In [14]:
print(points.shape)

(4, 10000, 3)


In [15]:
print(points[0].shape)

(10000, 3)


In [16]:
from SimAttention.network.augmentation import Batch_PointWOLF
aug_method = Batch_PointWOLF()

aug = aug_method(points)
print(aug.shape, type(aug))

(4, 10000, 3) <class 'numpy.ndarray'>


### Augmentation Method passed!

### 开始检查crop方法

In [17]:
from SimAttention.utils.crops import b_fps
sub1 = b_fps(aug, 1024)
print(sub1.shape)

(4, 1024, 3)


In [18]:
from SimAttention.utils.crops import b_get_slice
slice1 = b_get_slice(aug, 1, 1, 1024) 
print(slice1.shape)

(4, 1024, 3)


In [19]:
from SimAttention.utils.crops import b_get_cube
cube1 = b_get_cube(aug, 0.2, 1024)
print(cube1.shape)

(4, 1024, 3)


In [20]:
from SimAttention.utils.crops import b_get_sphere
sphere1 = b_get_sphere(aug, 0.1, 1024)
print(sphere1.shape)

(4, 1024, 3)


### crops method 验证完成

### Encoder的输入输出测试

In [21]:
from SimAttention.network.encoder import PCT_Encoder

net_encoder = PCT_Encoder()
sphere1 = torch.Tensor(sphere1)
out_put = net_encoder(sphere1)
print(out_put.shape)

torch.Size([4, 1, 1024])


### SimAttention测试

In [22]:
from SimAttention.utils.provider import set_requires_grad, get_attention_feature, get_target_encoder
online_encoder = net_encoder
target_encoder = get_target_encoder(online_encoder)

out_put = target_encoder(sphere1)
print(out_put.shape)

torch.Size([4, 1, 1024])


In [23]:
#### get_attention_feature 测试
q = torch.ones((4, 1, 1024))
kv = torch.ones((4, 6, 1024))
a = get_attention_feature(q, kv)
a.shape

torch.Size([4, 1024])

In [24]:
# output encoder dim = [4, 1, 1024]
Q = torch.ones(4, 1024)
Q_p = Q.reshape(4, 1, -1)
Q_p.shape

torch.Size([4, 1, 1024])

In [25]:
sub1 = torch.Tensor(sub1)
sub2 = sub1

slice1 = torch.Tensor(slice1)
slice2 = slice1

cube1 = torch.Tensor(cube1)
cube2 = cube1

sphere2 = sphere1
attention_feature_method = get_attention_feature

print(type(slice1))

<class 'torch.Tensor'>


In [26]:
from SimAttention.model import SimAttention

net = SimAttention(sub1, sub2, slice1, slice2, cube1, cube2, sphere1, sphere2,
                    online_encoder, target_encoder, attention_feature_method)
# loss = net(torch.Tensor(points))
# loss

### 整合

- sub之类的东西，而不是需要作为初始化参数
- 办法就是一个输入，然后其他的处理都是传入一个函数名作为参数

In [27]:
from SimAttention.model import SimAttention_2

net_2 = SimAttention_2(aug_method, b_fps, b_get_slice, b_get_cube, b_get_sphere, 
                       online_encoder, attention_feature_method)

loss_2 = net_2(torch.Tensor(points))
loss_2

tensor([ 1.7881e-06,  1.4305e-06,  1.3113e-06, -2.3842e-07],
       grad_fn=<RsubBackward1>)

In [29]:
from SimAttention.model import SimAttention_3
net_3 = SimAttention_3(aug_method, b_fps, b_get_slice, b_get_cube, b_get_sphere, 
                       online_encoder, attention_feature_method)

loss_3 = net_3(torch.Tensor(points))
loss_3

tensor([ 7.6294e-06, -7.1526e-07, -1.4305e-06,  3.6836e-05],
       grad_fn=<AddBackward0>)

### loss
- 应该上下两支交换aug之后 再次进行计算，得到一组差，最后将这两个差加起来;
- net_3 就是这么处理的，看上去的差值变大了；

### todo
- 写好train函数
- 重新设置平
